In [3]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import mysql.connector
import re
from dotenv import load_dotenv
import os

In [23]:
df_faire=pd.read_csv('faire_cleaned.csv')
df_paris_data=pd.read_csv('paris_data_cleaned.csv')
df_detailed_places=pd.read_csv('detailed_places_cleaned.csv')
data=pd.read_csv("data_cleaned.csv")
df_kf_places_paris=pd.read_csv('kf_places_paris_cleaned.csv')
df_parks_with_playground=pd.read_csv('parks_with_playground_cleaned.csv')

In [25]:
#Dropping innecesary columns
df_detailed_places=df_detailed_places.drop(columns=['rating','total_ratings'])
df_parks_with_playground=df_parks_with_playground.drop(columns=["Ancien nom de l'espace vert","Année de l'ouverture",])
df_faire=df_faire.drop(columns=['audience',"Type d'accès",])
df_paris_data=df_paris_data.drop(columns=['Name Arrondissement.1','Arrondissement'])
data=data.drop(columns=['rating','total_ratings','more_kids_friendly_weighted_sentiment','kids_friendly_sentiment'])
df_kf_places_paris=df_kf_places_paris.drop(columns=['rating',"arrondissement",'user_ratings_total','zip_code',])

In [47]:
#Connection to SQL
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")


conn = mysql.connector.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME
)
cursor = conn.cursor()

In [29]:
# Dataset to send to sql
datasets = {
    "detailed_places": "cleaned_ddetailed_kids_friendly_places_paris_with_reviews.csv",
    "parks_with_playground": "espaces_verts.csv",
    "faire": "qque-faire-a-paris-.csv",
    "paris_data": "ppopulation_paris_2021.csv",
    "data": "Gral_df_Scores.csv",
    "kf_places_paris": "extra_kfplaces_paris.csv"
}

#The columns expected
expected_columns = {
    "detailed_places": ['name', 'type', 'vicinity', 'formatted_address', 'website',
                        'phone_number', 'opening_hours', 'reviews', 'zipcode',
                        'unique_id', 'type_id', 'df_id', 'df_unique_id'],
    "parks_with_playground": ['unique_id', 'nom_espace_vert', 'typologie_espace_vert', 'categorie',
                              'zipcode', 'surface_calculee', 'superficie_totale_reelle', 'perimetre',
                              'site_villes', 'geo_shape', 'url_plan', 'geo_point', 'adresse', 'type',
                              'type_id', 'df_id', 'df_unique_id'],
    "faire": ['unique_id', 'url', 'titre', 'chapeau', 'description', 'description_date', 'nom_lieu',
              'adresse_lieu', 'zipcode', 'ville', 'coordonnees_geographiques', 'type_prix', 'groupe',
              'locale', 'month_debut', 'year_debut', 'month_fin', 'year_fin', 'type', 'type_id',
              'df_id', 'df_unique_id'],
    "paris_data": ['name_arrondissement', 'surface_ha', 'population_2020',
                   'density_2021_hab_km2', 'zipcode'],
    "data": ['name', 'vicinity', 'formatted_address', 'website', 'phone_number',
             'opening_hours', 'reviews', 'latitude', 'longitude', 'proche_park',
             'kids_mentions_count', 'zipcode', 'unique_id', 'kids_friendly', 'type',
             'type_id', 'df_id', 'df_unique_id'],
    "kf_places_paris": ['name', 'vicinity', 'type', 'latitude', 'longitude',
                        'zipcode', 'unique_id', 'type_id', 'df_id', 'df_unique_id']
}

# Founction to clen every csv
def clean_data(df):
    def remove_emojis(text):
        emoji_pattern = re.compile(
            "["                               
            "\U0001F600-\U0001F64F"  # Emoticons
            "\U0001F300-\U0001F5FF"  # Symboles and pictogrammes divers
            "\U0001F680-\U0001F6FF"  # Transports and symboles
            "\U0001F1E0-\U0001F1FF"  # Flgs
            "\U00002500-\U00002BEF"  # Other stuff
            "\U00002702-\U000027B0"  # Symboles divers
            "\U0001F900-\U0001F9FF"  # Emoji divers
            "\U0001FA70-\U0001FAFF"  # More emojis
            "\U00002600-\U000026FF"  # Symboles divers
            "]+", 
            flags=re.UNICODE
        )
        return emoji_pattern.sub(r'', text)

    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            df[column] = pd.to_numeric(df[column], errors="coerce").fillna(0)
        elif pd.api.types.is_string_dtype(df[column]):
            df[column] = df[column].astype(str).str.strip().replace("nan", "").fillna("")
            df[column] = df[column].apply(remove_emojis)

            if column == 'zipcode':
                df[column] = df[column].str[:10]
    return df

# Function to insert the dato to sql
def insert_data(df, table_name):
    placeholders = ", ".join(["%s"] * len(df.columns))
    columns = ", ".join([f"`{col}`" for col in df.columns])

    # DUPLICATE KEY UPDATE
    update_clause = ", ".join([f"`{col}`=VALUES(`{col}`)" for col in df.columns if col != 'unique_id'])

    query = f"""
    INSERT INTO `{table_name}` ({columns}) VALUES ({placeholders})
    ON DUPLICATE KEY UPDATE {update_clause};
    """

    for _, row in df.iterrows():
        cursor.execute(query, tuple(row))
    conn.commit()

# for each dataset
for table_name, file_path in datasets.items():
    print(f"Traitement du fichier : {file_path} pour la table : {table_name}")

    
    df = pd.read_csv(file_path, encoding="utf-8", index_col=False)

  
    if table_name in expected_columns:
        actual_columns = df.columns.tolist()
        expected = expected_columns[table_name]

        for col in set(expected) - set(actual_columns):
            df[col] = None  
        df = df.reindex(columns=expected)

    # Cleaning the data
    df = clean_data(df)

    # Drop the duplicate in case
    if 'unique_id' in df.columns:
        df = df.drop_duplicates(subset='unique_id', keep='first')

    # Get the data into sql
    insert_data(df, table_name)

print("Importation finish.")
cursor.close()
conn.close()

Traitement du fichier : cleaned_ddetailed_kids_friendly_places_paris_with_reviews.csv pour la table : detailed_places
Traitement du fichier : espaces_verts.csv pour la table : parks_with_playground
Traitement du fichier : qque-faire-a-paris-.csv pour la table : faire
Traitement du fichier : ppopulation_paris_2021.csv pour la table : paris_data
Traitement du fichier : Gral_df_Scores.csv pour la table : data
Traitement du fichier : extra_kfplaces_paris.csv pour la table : kf_places_paris
Importation finish.


In [ ]:
#Normalized data for bigquery

df_faire=pd.read_excel(r'C:\Users\Tanguy\Documents\IRONHACK\Week8\Kids friendly projet\mysql_kf\normalized\qque-faire-a-paris-.xlsx')
data=pd.read_excel(r'C:\Users\Tanguy\Documents\IRONHACK\Week8\Kids friendly projet\mysql_kf\normalized\Gral_df_Scores.xlsx')
df_kf_places_paris=pd.read_excel(r'C:\Users\Tanguy\Documents\IRONHACK\Week8\Kids friendly projet\mysql_kf\normalized\extra_kfplaces_paris.xlsx')
df_parks_with_playground=pd.read_excel(r'C:\Users\Tanguy\Documents\IRONHACK\Week8\Kids friendly projet\mysql_kf\normalized\espaces_verts.xlsx')

In [ ]:
df_normalized = pd.concat([df_faire, data, df_kf_places_paris, df_parks_with_playground], axis=0, ignore_index=True, sort=False)
df_normalized.to_excel('normalized_df.xlsx')
df_normalized.to_csv('normalized_df.csv')